# DS-SF-34 | Unit Project | 3 | Machine Learning Modeling and Executive Summary | Starter Code

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.  You will summarize and present your findings and the methods you used.

In [652]:
import os

import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import linear_model

import scipy.stats as stats

In [562]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'dataset-ucla-admissions.csv'))
df.dropna(inplace = True)

df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether an applicant was admitted.

In [563]:
pd.crosstab(df.prestige,
        df.admit, dropna = False)



admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part B.  Feature Engineering

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [595]:
p =  pd.get_dummies(df.prestige, prefix = 'prestige')

In [596]:
p

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
395,0,1,0,0
396,0,0,1,0
397,0,1,0,0
398,0,1,0,0


> ### Question 3.  How many of these binary variables do we need for modeling?

In [597]:
pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


We need binary variables from prestige 2.0, 3.0, and 4.0

<font color=blue>Correcto

> ### Question 4.  Why are we doing this?

We're going to treat prestige_1 as our baseline and exclude it from our fit. This is done to prevent multicollinearity









> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [643]:
cols_to_keep = ['admit', 'gre', 'gpa']

In [644]:
df = df[cols_to_keep].join(p.ix[:, 'prestige_2':])
df_1 = df[cols_to_keep].join(p.ix[:, 'prestige_1':])

In [600]:
df

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1
...,...,...,...,...,...,...
395,0,620.0,4.00,1,0,0
396,0,560.0,3.04,0,1,0
397,0,460.0,2.63,1,0,0
398,0,700.0,3.65,1,0,0


In [601]:
df['intercept']= 1.0

## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether an applicant was admitted.

In [649]:
pd.crosstab(df_1['admit'],df_1['prestige_1.0'], rownames=['admit'])

prestige_1.0,0,1
admit,,
0,243,28
1,93,33


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [605]:
p1_admit = 33.0/(28.0+33.0)

odds_1_Admit = p1_admit / (1 - p1_admit)

odds_1_Admit

1.1785714285714288

> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [606]:
p234_admit = 94.0 / 336.0

p234_admit
odds_234_admit = p234_admit / (1 - p234_admit)

odds_234_admit

0.3884297520661157

<font color=blue>94, but yeah totally the right idea

> ### Question 9.  Finally, what's the odds ratio?

In [97]:
odds_ratio = odds_1_Admit / odds_234_admit

odds_ratio

3.0341945288753807

> ### Question 10.  Write this finding in a sentence.

The odds of admitting to UCLA is 200% greater coming from a #1 ranked college compared to someone who did not attend a #1 ranked college. 

<font color=blue>Perfect

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentence.

In [607]:
p_4_admit = 12.0/(67)

odds_4_admit = p_4_admit / (1 - p_4_admit)
odds_4_admit

0.21818181818181817

In [179]:
#oddds ratio

odds_4_admit / p234_admit

0.7798839458413926

<font color=blue> You forgot to write your finding in a sentence ;(

Answer: TODO

## Part D. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model predicting admission into UCLA using `gre`, `gpa`, and the `prestige` of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [608]:
train_cols = df.columns[1:]
      # Index([gre, gpa, prestige_2, prestige_3, prestige_4], dtype=object)
        

In [609]:
logit = sm.Logit(df['admit'], df[train_cols])

In [610]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


In [611]:
model = linear_model.LogisticRegression().\
    fit(X, df.admit)


> ### Question 13.  Print the model's summary results.

In [616]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 12 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        12:53:46   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept       -3.8769      1.142     -3.393      0.001        -6.116    -1.638
================================================================================
"""

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

In [614]:
conf = result.conf_int()

conf.columns = ['2.5%', '97.5%']

print np.exp(conf)

                  2.5%     97.5%
gre           1.000074  1.004372
gpa           1.136120  4.183113
prestige_2.0  0.272168  0.942767
prestige_3.0  0.133377  0.515419
prestige_4.0  0.093329  0.479411
intercept     0.002207  0.194440


In [615]:
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [617]:
# The exponent of the coefficients are the oddds ratio of the different features. GRE is 1.002, GPA is 2.18. 

> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

In [618]:
np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

There is an inverse relationship between prestige of school and admission rate. This means we can expect the odds of being admitted to decrease by 50% of the prestige of a school is 2. 

<font color=blue>Your interpretation is misleading. There is an inverse relationship between admission and prestige_#, but that is an artifact of indexing; More prestige >> greater likelihood of admission.

> ### Question 16.  Interpret the odds ratio of `gpa`.

GPA has a odds ratio of 2.18. Meaning that each 1.0 increase in GPA increases the probability by 118% of admission. 

<font color=blue> Odds, not probability.

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [619]:
gres = np.linspace(df['gre'].min(), df['gre'].max(), 10)

gpas = np.linspace(df['gpa'].min(), df['gpa'].max(), 10)



In [620]:
  def cartesian(arrays, out=None):
      arrays = [np.asarray(x) for x in arrays]
      dtype = arrays[0].dtype

      n = np.prod([x.size for x in arrays])
      if out is None:
          out = np.zeros([n, len(arrays)], dtype=dtype)

      m = n / arrays[0].size
      out[:,0] = np.repeat(arrays[0], m)
      if arrays[1:]:
          cartesian(arrays[1:], out=out[0:m,1:])
          for j in xrange(1, arrays[0].size):
              out[j*m:(j+1)*m,1:] = out[0:m,1:]
      return out

In [621]:
  # enumerate all possibilities
  combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
  # recreate the dummy variables
  combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
  dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
  dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

  # keep only what we need for making predictions
  cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
  combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])


In [622]:
  # make predictions on the enumerated dataset
  combos['admit_pred'] = result.predict(combos[train_cols])

In [623]:
combos.tail(5)

,gre,gpa,prestige,intercept,prestige_2.0,prestige_3.0,prestige_4.0,admit_pred
395,800.0,3.806667,4.0,1.0,0,0,1,0.334286
396,800.0,4.000000,1.0,1.0,0,0,0,0.734040
397,800.0,4.000000,2.0,1.0,1,0,0,0.582995
398,800.0,4.000000,3.0,1.0,0,1,0,0.419833
399,800.0,4.000000,4.0,1.0,0,0,1,0.368608


Answer: Assuming a student has a GRE score of 800 and GPA of 4.0, his/her probability of being admitted to UCLA from the different tiers of school are the following.

- Tier 1 : 73%
- Tier 2 : 58%
- Tier 3 : 42%
- Tier 4 : 37% 

<font color=blue>Totally correct. Another, easier way to make predictions on hypothetical data would be:

In [ ]:
result.predict([800, 4.0, 1, 1, 0, 0])

<font color=blue>I'm not totally sure what your functions above were accomplishing. You should write a short docstring when you know somebody else is going to be reading your code.

## Part E. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [624]:
X = df[ ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0','prestige_4.0'] ]
c = df.admit 

model = linear_model.LogisticRegression().\
    fit(X, c)

In [625]:
print model.intercept_
print model.coef_

[-1.81701706]
[[ 0.00178497  0.23229458 -0.60347467 -1.17214957 -1.37729795]]


> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [626]:
np.exp(model.coef_)

array([[ 1.00178657,  1.26149128,  0.546908  ,  0.3097005 ,  0.25225925]])

Take the exponent of the coefficeints to get oddds ratio:
    GRE: 1.00178657
    GPA: 1.26149128
    Prestige_2.0: 0.546908
    Prestige_3.0: 0.3097005
    Prestige_4.0: 0.252

> ### Question 20.  Again, assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [627]:
predict_X_tier2 = [ [800, 4.0,1,0,0]]
print model.predict_proba(predict_X_tier2)

[[ 0.51582873  0.48417127]]


In [628]:
# Probability of a student with GRE 800 AND GPA of 4.0 coming from tier-2 institution has 48.4% of admission

In [629]:
predict_X_tier3 = [ [800, 4.0,0,1,0]]
print model.predict_proba(predict_X_tier3)

[[ 0.6529453  0.3470547]]


In [630]:
# Probability of a student with GRE 800 AND GPA of 4.0 coming from tier-3 institution has 34.7% of admission"

In [631]:
predict_X_tier4 = [ [800, 4.0,0,0,1]]
print model.predict_proba(predict_X_tier4)

[[ 0.6978666  0.3021334]]


In [632]:
# Probability of a student with GRE 800 AND GPA of 4.0 coming from tier-4 institution has 30.2% of admission"

## Part F.  Executive Summary

> ## Question 21.  Introduction
>
> Write a problem statement for this project.

Our goal will be to identify the various factors that may influence admission into graduate school at UCLA. Those factors are GRE, GPA, and Prestige of the Undergraduate instiution. We know that there is a positive correlation between GRE, GPA and admit rate. GRE and GPA are continuous variables while Prestige is categorical and we would like to convert categorical variables to binary variables in prestige to find out, holding GRE & GPA constant, what is the probability of admission at specific levels of prestige. 

> ## Question 22.  Dataset
>
> Write up a description of your data and any cleaning that was completed.

We analyzed 397 row sof admission data. Each row represents a unique applicant. All rows contain continuous variables for GPA, GRE. Admission categorical and  is either 1 (admit) or 0 (not admit). Other categorical variable is prestige, which is either any number 1-4. 

Before we preformed logistical regression, we created dummies for prestige to add to our existing database. This is so that we can get binary indicator variables for each category/option in the prestige column.

> ## Question 23.  Demo
>
> Provide a table that explains the data by admission status.

In [651]:
df.pivot_table(df,index=["admit"])

,gpa,gre,prestige_2.0,prestige_3.0,prestige_4.0
admit,,,,,
0,3.347159,573.579336,0.350554,0.343173,0.202952
1,3.489206,618.571429,0.420635,0.222222,0.095238


> ## Question 24.  Methods
>
> Write up the methods used in your analysis.

Main method of prediction was using logistical regression using STATS Model and SKILearn. 

In STATSModel, I used a logit function and separated the predictor variable (admit_rate) from the training variables (GRE, GPA, Prestige_2.0, Prestige_3.0, and Prestige_4.0)



> ## Question 25.  Results
>
> Write up your results.

We have found that holding all variables constant, the probability of admission decreases considerably with each level of prestige of the undergraduate institution. The variance 

Holding all other variables constant, the probability of admission that is closer to 50% is from a tier-2 institution.

There is a difference between probability from using STATSMODEL and SKLEARN in that results from SKLEARN is roughly 5% lower in probability for each category of prestige compared to STATSMODEL. 

> ## Question 26.  Visuals
>
> Provide a table or visualization of these results.

In [639]:
pd.pivot_table(df,index=["admit"])

,gpa,gre,intercept,prestige_2.0,prestige_3.0,prestige_4.0
admit,,,,,,
0,3.347159,573.579336,1.0,0.350554,0.343173,0.202952
1,3.489206,618.571429,1.0,0.420635,0.222222,0.095238


> ## Question 27.  Discussion
>
> Write up your discussion and future steps.

Discussion: It will be interesting to find out if we hold prestige constant, what will be the probability of admission at different rates 
<font color=blue>Not sure what this means.</font>

Future Steps:  There are certainly other methods that can be discussed to analyze the data. We can perform regularization to determine if the model is too complex or if the model has an overfit problem. 

